In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from game import Game
from player import Player

from lukasz_151930 import Lukasz151930,Card_status

from naive import Naive

C:\Users\Łukasz\AppData\Roaming\Python\Python39\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [3]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(13, 1), (11, 0), (14, 1), (12, 0), (11, 3), (12, 3), (14, 0), (9, 1)]
[(10, 1), (13, 0), (13, 3), (11, 2), (10, 0), (10, 3), (13, 2), (10, 2)]


In [4]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [5]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(stats_moves[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class,repeats = 1000,log=False):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]

    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = log)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = log)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = log):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [6]:
strategy = [["Naive", "Naive", Naive],
            ["Łukasz", "Łukasz", Lukasz151930],
            ["Random", "RANDOM", RandomPlayer]
           ]

In [7]:
#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2],1000,False)
        full_results[A][B] = results
        


==== 0 Naive
1 Łukasz
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
DRAW
2 Random
==== 1 Łukasz
2 Random
DRAW
==== 2 Random


In [8]:
full_results

[[None,
  [[2, 998],
   [20, 20],
   [11.78, 12.246],
   [15.117, 0.087],
   0.796,
   [9.156, 0.204],
   [0.0, 1.905],
   [0.484, 0.011],
   [0, 0],
   0,
   20],
  [[990, 10],
   [0, 0],
   [17.261, 16.788],
   [0.16, 15.11],
   0.73,
   [14.881, 8.56],
   [0.016, 0.004],
   [0.455, 9.513],
   [0, 0],
   0,
   0]],
 [None,
  None,
  [[978, 22],
   [1, 1],
   [17.987, 17.526],
   [0.235, 13.724],
   2.041,
   [3.13, 7.622],
   [2.616, 0.015],
   [0.014, 9.712],
   [0, 0],
   0,
   1]],
 [None, None, None]]

Simple stats

In [9]:
def printMatrix(full_results, stat):
    print(STAT_NAMES[stat])
    S = " "
    for s in strategy: S += (str(s[1]) + " " )
    print(S)
    for A in range(len(strategy)):
        print(A)
        S = str(strategy[A][1]) + " "
        for B in range(len(strategy)):
            if A == B: S += "- "
            elif A < B:
                S += str(full_results[A][B][stat][0]) + " "
            else:
                S += str(full_results[B][A][stat][1]) + " "
        print(S)
for a in ANALYZE_STATS:
    printMatrix(full_results, a)
    print()



Wins
 Naive Łukasz RANDOM 
0
Naive - 2 990 
1
Łukasz 998 - 978 
2
RANDOM 10 22 - 

Draws
 Naive Łukasz RANDOM 
0
Naive - 20 0 
1
Łukasz 20 - 1 
2
RANDOM 0 1 - 

Moves
 Naive Łukasz RANDOM 
0
Naive - 11.78 17.261 
1
Łukasz 12.246 - 17.987 
2
RANDOM 16.788 17.526 - 

Cards
 Naive Łukasz RANDOM 
0
Naive - 15.117 0.16 
1
Łukasz 0.087 - 0.235 
2
RANDOM 15.11 13.724 - 

Checks
 Naive Łukasz RANDOM 
0
Naive - 9.156 14.881 
1
Łukasz 0.204 - 3.13 
2
RANDOM 8.56 7.622 - 

Draw Decisions
 Naive Łukasz RANDOM 
0
Naive - 0.0 0.016 
1
Łukasz 1.905 - 2.616 
2
RANDOM 0.004 0.015 - 

Cheats
 Naive Łukasz RANDOM 
0
Naive - 0.484 0.455 
1
Łukasz 0.011 - 0.014 
2
RANDOM 9.513 9.712 - 

Errors
 Naive Łukasz RANDOM 
0
Naive - 0 0 
1
Łukasz 0 - 0 
2
RANDOM 0 0 - 



In [10]:
a= np.array([[1,0,0],
         [0,1,1],
         [1,0,1]])

In [11]:
a * np.arange(1,4)

array([[1, 0, 0],
       [0, 2, 3],
       [1, 0, 3]])

In [12]:
l = Lukasz151930("lucas")

In [13]:
np.mean(np.arange(1,7))

3.5

In [14]:
def find_pure_nash(payoff_matrix:np.array):
    nash_equilibria = []

    for i in range(payoff_matrix.shape[0]):
        for j in range(payoff_matrix.shape[1]):
            if np.all(payoff_matrix[:, j, 0] <= payoff_matrix[i, j, 0]) and np.all(payoff_matrix[i, :, 1] <= payoff_matrix[i, j, 1]):
                nash_equilibria.append((i, j))
    
    return nash_equilibria

In [15]:
def mixed_strats(payoff:np.array):
    #A = np.vstack((payoff[0,:,0]-payoff[1,:,0],payoff[0,:,1]-payoff[1,:,1]))
    #b = np.array([payoff[0,0,0]-payoff[0,1,0],payoff[0,0,1]-payoff[1,0,1]])
    a11,a12,a21,a22 = payoff[:,:,0].flatten()
    b11,b12,b21,b22 = payoff[:,:,1].flatten()

    # A = np.array([[a[0,0]-a[1,0],a[0,1]-a[1,1]],
    #             [b[0,0]-b[0,1],b[1,0]-b[1,1]]])
    # c = np.array([a[1,0]-a[1,1],b[0,1]-b[1,1]])

    # probs = np.linalg.solve(A, c)
    # print(probs)
    p = (a22-a12)/(a11-a21-a12+a22)
    #if np.all(probs >= 0) and np.all(probs <= 1):
    q = (b22-b12)/(b11-b21-b12+b22)
    return (q, 1 - q),(p, 1 - p)

In [16]:
payoff = np.array([
    [[0,1],[2,0]],
    [[3,0],[0,0]]
])

find_pure_nash(payoff)

[(1, 0)]

In [17]:
mixed_strats(payoff)

((0.0, 1.0), (0.4, 0.6))

In [18]:
payoff[0,:,0]-payoff[1,:,0]

array([-3,  2])

In [19]:
payoff[:,:,0].flatten() 

array([0, 2, 3, 0])

In [20]:
payoff[:,0,0] - payoff[0,:,0]

array([0, 1])